# 基于检索的问答机器人
## 实现方式
QQ匹配：即从索引库中选出用户问题最相似的问题，然后给出对应的答案为输出结果。

## step1 读取数据

In [1]:
import pandas as pd

ds = pd.read_csv('../dataset/law_faq.csv')
ds.head()

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


## step2  加载模型

In [2]:
from Dualmodel import DualModel
dualmodel = DualModel.from_pretrained('dual_model/checkpoint-750')

In [3]:
model = dualmodel.cuda()
model.eval()

DualModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

## step3 加载Tokenizer

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-macbert-base')
tokenizer

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hfl/chinese-macbert-base/resolve/main/tokenizer_config.json (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1147)')))"), '(Request ID: e77271e0-1b8e-45e7-9674-2b412b0d71e2)')' thrown while requesting HEAD https://huggingface.co/hfl/chinese-macbert-base/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hfl/chinese-macbert-base/resolve/main/tokenizer_config.json (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1147)')))"), '(Request ID: 89478f61-02ec-46bb-b88b-0bfbed509faf)')' thrown while requesting HEAD https://huggingface.co/hfl/chinese-macbert-base/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

## step4 将问题编码为向量

In [5]:
questions = ds['title'].to_list()
questions[:5], len(questions)

(['在法律中定金与订金的区别订金和定金哪个受',
  '盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体',
  '非法微整形机构构成非法经营罪吗',
  '入室持刀行凶伤人能不能判刑',
  '对交通事故责任认定书不服怎么办，交通事故损'],
 18213)

In [7]:
import torch
from tqdm import tqdm
vectors = []
with torch.inference_mode():
    for i in tqdm(range(0, len(questions), 32)):
        batch_sens = questions[i:i+32]
        inputs = tokenizer(batch_sens, return_tensors="pt", padding=True, max_length=128, truncation=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        vector = model.bert(**inputs)[1]  #这里取[1]是因为bert输出的[0]是token的隐藏向量
        vectors.append(vector)
vectors = torch.concat(vectors,dim=0).cpu().numpy()
vectors.shape

100%|██████████| 570/570 [00:17<00:00, 32.76it/s]


(18213, 768)

## step5 创建索引

In [8]:
import faiss

index = faiss.IndexFlatIP(768)  #创建基于内积的索引 这里768是bert的输出维度
faiss.normalize_L2(vectors)     #对向量归一化，使内积等价于余弦相似度
index.add(vectors)              #将向量批量添加到索引

## step6 对问题进行向量编码

In [36]:
question = "三方协议是什么"
input = tokenizer(question, return_tensors="pt", padding=True, max_length=128, truncation=True)
input = {k: v.to(model.device) for k, v in input.items()}
vector = model.bert(**input)[1]
q_vector = vector.cpu().detach().numpy()
q_vector.shape

(1, 768)

## step7 向量匹配

In [37]:
faiss.normalize_L2(q_vector)
result = index.search(q_vector, 10)
result

(array([[0.7915735 , 0.7633049 , 0.7629776 , 0.7559693 , 0.7412051 ,
         0.7407516 , 0.7368232 , 0.73529357, 0.725943  , 0.7257179 ]],
       dtype=float32),
 array([[14176,  7289, 12094,  8387, 18054, 12019,  7377,   571,  7612,
         11423]]))

In [38]:
scores, indexes = result
scores = scores.squeeze(0)
indexes = indexes.squeeze(0)
scores, indexes

(array([0.7915735 , 0.7633049 , 0.7629776 , 0.7559693 , 0.7412051 ,
        0.7407516 , 0.7368232 , 0.73529357, 0.725943  , 0.7257179 ],
       dtype=float32),
 array([14176,  7289, 12094,  8387, 18054, 12019,  7377,   571,  7612,
        11423]))

In [39]:
match_questions = []
answers = []
for i in range(10):
    match_questions.append(questions[indexes[i]])
    answers.append(ds['reply'][indexes[i].tolist()])
match_questions, answers

(['不能转让的债权债务指的是什么',
  '变更抚养关系的法律依据是什么',
  '刑法中无罪推定是指什么，罪刑法定是指什么',
  '撤销权的诉讼时效是怎样规定的',
  '借贷合同纠纷法律上具体是怎样规定的呢',
  '法律对继承权诉讼时效是怎样规定的',
  '五险一金是什么',
  '立遗嘱把财产留给谁的规定的内容是什么',
  '二审程序和一审程序的区别是什么',
  '婚前保证书具有哪些法律效力'],
 ['二、根据合同法第七十九条的规定，不得转让的债权的情况有：1、当事人约定不得转让的，要求这种约定须在转让之前订立，否则，该约定无效。2、依合同性质不得转让的，这类债权要么与债权人的人身有不可分割的关系，要么基于债权人与债务人间的信任关系产生，如扶养请求权、雇主对于雇员、委托人对于代理人的债权、不作为债权等。3、法律规定不得转让的。国家以法律禁止转让的，违反禁止性规定的，合同当然无效。如不良资产、国防、军工等涉及安全和敏感信息的债权等。《合同法》第八十四条规定“债务人将合同的义务全部或者部分转移给第三人的，应当经债权人同意。”',
  '判决子女抚养权的归属，是以有利于保护子女的利益为原则的，如果出现了不利于子女健康成长的情形，就应变更子女的抚养关系。因此，离婚后，子女抚养关系可以变更。法律规定有下列情形之一的，一方有权要求变更子女抚养关系：（1）与子女共同生活的一方因患严重疾病或因伤残无力继续抚养子女的；（2）与子女共同生活的一方不尽抚养义务或有虐待子女行为，或其与子女共同生活对子女身心健康确有不利影响的；（3）十周岁以上未成年子女，愿随另一方生活，该方又有抚养能力的；（4）有其他正当理由需要变更的。此外，父母双方协议变更子女抚养关系的，应予准许。',
  '无罪推定是指任何人在未经依法判决有罪之前，应视其无罪。法无明文规定不为罪和法无明文规定不处罚是罪刑法定的基本含义',
  '撤销权是除斥期间，不是诉讼时效。合同法第五十五条【撤销权的消灭】有下列情形之一的，撤销权消灭：（一）具有撤销权的当事人自知道或者应当知道撤销事由之日起一年内没有行使撤销权；……',
  '借款合同(*间借贷)常用法律法规及司法解释：1、《中华人*共和**法通则》(1986年4月12日)第八十五条合同是当事人之间设立、变更、终止*事关系的协议。依法成立的合同，受法律保护。第九

后续还可以将 选择的topk match_question和输入的question进行交互匹配 提高回答准确率